# FastAPI and Streamlit Tutorial 

## Introduction to APIs
An API (Application Programming Interface) is like a waiter in a restaurant. Just as a waiter takes your order, goes to the kitchen (backend), and brings back your food, an API takes requests from users, processes them in the backend, and returns responses. APIs allow different software applications to communicate with each other in a structured way.

## What is FastAPI?
FastAPI is a modern Python web framework for building APIs. It's known for being:

- Fast to run (high performance)
- Fast to code (easy to use)
- Fewer bugs (automatic validation)
- Automatic documentation

In [ ]:
!pip install fastapi uvicorn streamlit pandas python-multipart

### Creating the Fast Backend

In [4]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import uvicorn
import json
from datetime import datetime

In [ ]:
# Define the data model
class Student(BaseModel):
    id: int
    name: str
    age: int
    grade: float
    enrollment_date: str

# Initialize with some test data
students = [
    Student(
        id=1,
        name="John Doe",
        age=20,
        grade=85.5,
        enrollment_date="2024-01-15"
    ),
    Student(
        id=2,
        name="Jane Smith",
        age=19,
        grade=92.0,
        enrollment_date="2024-01-16"
    )
]
# Initialize FastAPI app
app = FastAPI(title="Student Management System")


### REST API 

REST (Representational State Transfer) is a design style for building APIs that interact with resources like users or products. It uses stateless communication, meaning each request is independent, and the server doesn’t remember previous interactions; the client must include all necessary information in every request. REST also uses resource-based URLs, where each URL represents a specific resource (e.g., /users/123 for a user with ID 123) and actions are defined using standard HTTP methods like GET or POST.

### API endpoints 

API endpoints are specific URLs in a REST API that represent resources or actions, allowing clients to access or manipulate data. For example, GET /users retrieves user data, while POST /users creates a new user.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import uvicorn
import json
from datetime import datetime



# API Endpoints

@app.get("/")
def read_root():
    return {"message": "Welcome to Student Management System API"}

@app.get("/students/", response_model=List[Student])
def get_all_students():
    return students

@app.get("/students/{student_id}")
def get_student(student_id: int):
    student = next((s for s in students if s.id == student_id), None)
    if student is None:
        raise HTTPException(status_code=404, detail="Student not found")
    return student

@app.post("/students/", response_model=Student)
def add_student(student: Student):
    # Check if student ID already exists
    if any(s.id == student.id for s in students):
        raise HTTPException(status_code=400, detail="Student ID already exists")
    students.append(student)
    return student

@app.get("/statistics/")
def get_statistics():
    if not students:
        return {
            "average_grade": 0,
            "total_students": 0,
            "average_age": 0
        }
    
    return {
        "average_grade": sum(s.grade for s in students) / len(students),
        "total_students": len(students),
        "average_age": sum(s.age for s in students) / len(students)
    }



### How to run FAST API

To run this, put it in a single file called main.py and call

uvicorn main:app --reload

## Streamlit
What is Streamlit?
Streamlit is a Python library that makes it easy to create web applications for data science and machine learning.

It turns data scripts into shareable web apps in minutes, requiring minimal frontend experience.

### Project Overview
We'll build a simple Student Management System where we can:

- Add student information
- View all students
- Search for students by ID
- Calculate class statistics

#### Creating the Streamlit Frontend
Create a new file called frontend.py with the following details

In [ ]:
import streamlit as st
import requests
import json
from datetime import datetime
import pandas as pd

# Set the API URL
API_URL = "http://localhost:8000"

st.title("Student Management System")

# Sidebar for navigation
page = st.sidebar.selectbox("Choose a page", ["View Students", "Add Student", "Search Student", "Statistics"])

if page == "View Students":
    st.header("All Students")
    
    # Fetch all students
    response = requests.get(f"{API_URL}/students/")
    if response.status_code == 200:
        students = response.json()
        if students:
            df = pd.DataFrame(students)
            st.dataframe(df)
        else:
            st.info("No students found in the system.")
    else:
        st.error("Failed to fetch students data.")

elif page == "Add Student":
    st.header("Add New Student")
    
    # Create input form
    with st.form("add_student_form"):
        student_id = st.number_input("Student ID", min_value=1, step=1)
        name = st.text_input("Name")
        age = st.number_input("Age", min_value=15, max_value=100, step=1)
        grade = st.number_input("Grade", min_value=0.0, max_value=100.0, step=0.1)
        enrollment_date = st.date_input("Enrollment Date")
        
        submit_button = st.form_submit_button("Add Student")
        
        if submit_button:
            student_data = {
                "id": student_id,
                "name": name,
                "age": age,
                "grade": grade,
                "enrollment_date": enrollment_date.strftime("%Y-%m-%d")
            }
            
            response = requests.post(
                f"{API_URL}/students/",
                json=student_data
            )
            
            if response.status_code == 200:
                st.success("Student added successfully!")
            else:
                st.error(f"Error: {response.json()['detail']}")

elif page == "Search Student":
    st.header("Search Student")
    
    student_id = st.number_input("Enter Student ID", min_value=1, step=1)
    if st.button("Search"):
        response = requests.get(f"{API_URL}/students/{student_id}")
        if response.status_code == 200:
            student = response.json()
            st.write("Student Information:")
            st.json(student)
        else:
            st.error("Student not found.")

elif page == "Statistics":
    st.header("Class Statistics")
    
    response = requests.get(f"{API_URL}/statistics/")
    if response.status_code == 200:
        stats = response.json()
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.metric("Total Students", f"{stats['total_students']}")
        with col2:
            st.metric("Average Grade", f"{stats['average_grade']:.2f}")
        with col3:
            st.metric("Average Age", f"{stats['average_age']:.1f}")

### How to run the streamlit app

In a new terminal, start the streamlit frontend

streamlit run frontend.py